# Jeu d'échecs par renforcement I


## Introduction

Le jeu d'échecs a toujours été un cas emblématique des algorithmes d'intelligence artificielle, d'une part parce que :

1. Ses règles sont bien établies et non ambiguës ;
2. La dimension du problème rend impossible une résolution « fonctionnelle »
3. On a longtemps pensé (années 196x - 197x) que c'était un problèpe simple à résoudre par une approche exploratoire.


L'objectif est d'implémenter un jeu d'échecs en choisissant une méthode d'apprentissage par renforcement, c'est-à-dire une méthode qui ne nécessite pas de modèle pré-établi.

Dans notre cas, nous n'aurons donc pas besoin de collecter une bibliothèque de parties qu nous permettraient de montrer au réseau de neurones quelles sont celles dont la stratégie est gagnante et celles où elle est perdante.

L'apprentissage par renforcement (AR) va permettre au réseau d'apprendre quels sont les coups qui la « plus grande valeur »  dans un contexte donné et de déterminer le meilleur « chemin » vers le gain de la partie. 

D'une certaine manière, AR est l'héritier d'algorithmes classiques comme MinMax. Mais le problème de ces algorithmes est que la valeur de certaines décisions est déterminée _a priori_ et qu'il faut écrire des fonctions, dites **heuristiques**, qui permettent au programme de calculer la valeur attribuée à une décision. Cequi est naturellement un problème très difficile. AR vise à résoudre ce problème en laissant le réseau de neurones apprendre par lui-même la fonction heuristique. En conséquence, celle-ci pourra dépendre d'un nombre arbitrairement grand de paramètres.

## Etape 1 : Environnement

Nos avons besoin, naturellement, de représenter les éléments du jeu : 
- l'échiquier
- les pièces
- la partie


### L'échiquier

L'échiquier nous permet simplment de définir un espace de déplacement pour les pièces, avec en particulier un point de départ et un point d'arrivée.

In [ ]:
import numpy as np

class Board :
    """
    Un échiquier
    """
    def __init__(self):
        # Position initiale
        self.state = (0, 0)
        # Position finale
        self.terminal_state = (7, 5)
        # Espace des récompenses
        # Tous les déplacements sont équivalents et reçoivent une récompense de -1
        self.reward_space = np.zeros(shape=(8, 8)) - 1

    def step(self, action):
        """
        Calcul de la récompense liée à un déplacement vers une nouvelle position
        
        :param action: Le déplacement opéré par un agent
        :type action:  tuple
        
        :return:       La récompsense pour le déplacement
        :rtype:        tuple
        """
        reward = self.reward_space[self.state[0], self.state[1]]
        if self.state == self.terminal_state:
            episode_end = True
            return 0, episode_end
        else:
            episode_end = False
            old_state = self.state
            new_state = (self.state[0] + action[0], self.state[1] + action[1])
            self.state = old_state if np.min(new_state) < 0 or np.max(new_state) > 7 else new_state
            return reward, episode_end

    def render(self):
        """
        Rendu visuel de l'échiquier dans le terminal
        """
        visual_row = ["[ ]", "[ ]", "[ ]", "[ ]", "[ ]", "[ ]", "[ ]", "[ ]"]
        visual_board = []
        for c in range(8):
            visual_board.append(visual_row.copy())
        visual_board[self.state[0]][self.state[1]] = "[S]"
        visual_board[self.terminal_state[0]][self.terminal_state[1]] = "[F]"
        self.visual_board = visual_board
        
# Test
board= Board()
board.render()
board.visual_board 


### Les pièces 

Une pièce du jeu d’échecs, qui peut être : Roi, Fou, Cavalier, Tour.

Pour le moment, nous avons juste besoin de définir l'espace des mouvements possibles de ces pièces.

In [ ]:
   
class Piece :
    """
    Une pièce du jeu d'écheccs
    """
    def __init__(self, piece="King"):
        self.piece = piece
        self.init_actionspace()


    def init_actionspace(self):
        """
        Définition des mouvements possibles pour chaque pièce du jeu
        """
        assert self.piece in ["King", "Rook", "Bishop", "Knight"], f"{self.piece} n'est pas une pièce autorisée"
        if self.piece == 'King':
            self.action_space = [(-1, 0),  # north
                                 (-1, 1),  # north-west
                                 (0, 1),  # west
                                 (1, 1),  # south-west
                                 (1, 0),  # south
                                 (1, -1),  # south-east
                                 (0, -1),  # east
                                 (-1, -1),  # north-east
                                 ]
        elif self.piece == 'Rook':
            self.action_space = []
            for amplitude in range(1, 8):
                self.action_space.append((-amplitude, 0))  # north
                self.action_space.append((0, amplitude))  # east
                self.action_space.append((amplitude, 0))  # south
                self.action_space.append((0, -amplitude))  # west
        elif self.piece == 'Knight':
            self.action_space = [(-2, 1),  # north-north-west
                                 (-1, 2),  # n-w-w
                                 (1, 2),  # s-w-w
                                 (2, 1),  # s-s-w
                                 (2, -1),  # s-s-e
                                 (1, -2),  # s-e-e
                                 (-1, -2),  # n-e-e
                                 (-2, -1)]  # n-n-e
        elif self.piece == 'Bishop':
            self.action_space = []
            for amplitude in range(1, 8):
                self.action_space.append((-amplitude, amplitude))  # north-west
                self.action_space.append((amplitude, amplitude))  # south-west
                self.action_space.append((amplitude, -amplitude))  # south-east
                self.action_space.append((-amplitude, -amplitude))  # north


### La partie

La partie est l'espace de calcul où évoluent les pièces du jeu d'échecs.

In [ ]:
 
class Reinforce :
    """
    Une partie
    """
    def __init__(self, agent, environment) :
        self.env = environment
        self.agent = agent

## Etape 2 : Déplacer les pièces

Dans un premier, nos allons apprendre à déplacer les pièces sur l'échiquier. Pour cela, nous posons les hypothèses suivantes :
- L'espace des états est un tableau à deux dimensions de 8 x 8
- Nous partons d'un état initial **S** aux coordonnées (0, 0)
- Nous choisissons unétat final **F** aux coordonnées (7,5)
- Chaque déplacement engendre une « récompense » de -1
- La meilleure stratégie est celle qui permet de se déplace se S à F en un minimum de coups

### Evaluation de la position

Le principe de AR est de faire en sorte qu'un « agent » cherche à optimiser ses récompenses. Le bu de l'apprentissage est donc de lui permettre de se diriger vers les positions qui lui permettront de maximiser celles-ci. Chaque position est associé à une certaine valeur. D'où :

- Une position (P) a une valeur (V) égale à la valeur (V’) de la position précédente (P’) plus la récompense (R)
- Comme plusieurs décisions sont possibles à chaque position (P), nous faisons la somme des valeurs multipliées par leur probabilité
- Les valeurs des positions suivantess sont pondérées par  un facteur (gamma) variant entre 0 et 1.
- La valeur pour unétat suivant est une estimation
- Evaluer une position est un processus récursif : on calcule une estimation basée sur d'autres estimations

Nous pouvons donc écrire la fonction d'évaluation suivante :

In [ ]:

class Reinforce :
    
    def evaluate_state(self, state, gamma=0.9, synchronous=True):
            """
            Calcule la valeur d'une position en fonction des valeurs des positions ultérieures (suivantes).
            Args:
                state: Les coordonnées de la position sur l'échiquier
                gamma: Le facteur de pondération
                synchronous: [...]

            Returns: La valeur de la position (étant donnée une certaine stratégie).

            """

            # Valuer optimale
            greedy_action_value = np.max(self.agent.policy[state[0], state[1], :])

            # Liste des mouvements possibles
            greedy_indices = [i for i, a in enumerate(self.agent.policy[state[0], state[1], :]) if
                              a == greedy_action_value]
            
            # Probabilité d'occurrence d'un mouvement
            prob = 1 / len(greedy_indices)
            
            # Valeur initiale
            state_value = 0
            
            # Itération sur tous les mouvements possibles
            for i in greedy_indices:
                # Actualisation de la position
                self.env.state = state
                # Calcul de la récompnse pour un mouvement possible
                reward, episode_end = self.env.step(self.agent.action_space[i])
                #if synchronous:
                    successor_state_value = self.agent.value_function_prev[self.env.state]
                #else:
                #    successor_state_value = self.agent.value_function[self.env.state]
                # Actualisation de la valeur de la position actuelle
                state_value += (prob * (reward + gamma * successor_state_value))
                
            # Valeur estimée de la position
            return state_value


### Evaluation de la stratégie

L'évaluation de la stratégie est simplement l'évaluation des positions pour tout l'espace des positions. Nous devons donc juste répéter la fonction `evalutate_state` pour toutes les positions possibles

In [ ]:

class Reinforce :
    
    def evaluate_policy(self, gamma=0.9, synchronous=True) :
        # Sauvegarde de la position précédente
        self.agent.value_function_prev = self.agent.value_function.copy()
        
        # Itération sur toutes les cases de l'échiquier
        for row in range(self.agent.value_function.shape[0]):
            for col in range(self.agent.value_function.shape[1]):
                self.agent.value_function[row, col] = self.evaluate_state((row, col), gamma=gamma, synchronous=synchronous)


Ceci étant posé, nouspouvons maintenant apprendre à reconnaître le meileur chemin en répétant l'évaluation jusqu'à ce que les valeurs soient stables :

In [ ]:
# initialisation du système
r = Reinfoce(Borad(), Piece(pice="King"))
# Définition d'un seuil 
epsilon=0.1
# Nombre d'itération maximal
k_max = 1000
# Ecart maximal
value_delta_max = 0
# Facteur de pondération
gamma = 1
# Evaluation synchrone
synchronous=True
for k in range(k_max):
    r.evaluate_policy(gamma=gamma,synchronous=synchronous)
    value_delta = np.max(np.abs(r.agent.value_function_prev - r.agent.value_function))
    value_delta_max = value_delta
    if value_delta_max < eps:
        print('converged at iter',k)
        break

### Optimisation de la stratégie

Maintenant que nous connaissons les valeurs des positions, ce qui nous intéresse est de guider notre « agent » vers la meilleure position possible, celle avec la valeur la plus grande. L'optimisation de la stratégie consiste simplement à instaurer un comportement « impatient » (ou « avide ») de l'agent, relativement à la valeur de chaque position.

In [ ]:

class Reinforce :
    
        def improve_policy(self):
        """
        Trouve la stratégie « impatiente » en regard de l'évaluation des positions.
        """

        self.agent.policy_prev = self.agent.policy.copy()
        for row in range(self.agent.action_function.shape[0]):
            for col in range(self.agent.action_function.shape[1]):
                for action in range(self.agent.action_function.shape[2]):
                    self.env.state = (row, col)  # reset state to the one being evaluated
                    reward, episode_end = self.env.step(self.agent.action_space[action])
                    successor_state_value = 0 if episode_end else self.agent.value_function[self.env.state]
                    self.agent.policy[row, col, action] = reward + successor_state_value

                max_policy_value = np.max(self.agent.policy[row, col, :])
                max_indices = [i for i, a in enumerate(self.agent.policy[row, col, :]) if a == max_policy_value]
                for idx in max_indices:
                    self.agent.policy[row, col, idx] = 1


De la même manière que précédemment, nous pouvons maintenant apprendre la meilleure trajectoire en itérant la fonction `improve_policy` jusqu'à ce que les valeurs soient stables :

In [ ]:
class Reinforce :
    
        def policy_iteration(self, eps=0.1, gamma=0.9, iteration=1, k=32, synchronous=True):
        """
        Trouve la stratégie optimale

        :param epsilon: 
        :type epsilon: float
        :para gamma: 
        :type gamma: float
        : param iteration:
        :type iteration: int
        :param k:
        :type k: int
        :param synchronous:
        :type synchronous: boolean

        :return:
        :rtype: void

        """
        policy_stable = True
        print("\n\n______iteration:", iteration, "______")
        print("\n policy:")
        self.visualize_policy()

        print("")
        value_delta_max = 0
        for _ in range(k):
            self.evaluate_policy(gamma=gamma, synchronous=synchronous)
            value_delta = np.max(np.abs(self.agent.value_function_prev - self.agent.value_function))
            value_delta_max = value_delta
            if value_delta_max < eps:
                break
        print("Value function for this policy:")
        print(self.agent.value_function.round().astype(int))
        action_function_prev = self.agent.action_function.copy()
        print("\n Improving policy:")
        self.improve_policy()
        policy_stable = self.agent.compare_policies() < 1
        print("policy diff:", policy_stable)

        if not policy_stable and iteration < 1000:
            iteration += 1
            self.policy_iteration(iteration=iteration)
        elif policy_stable:
            print("Optimal policy found in", iteration, "steps of policy evaluation")
        else:
            print("failed to converge.")